# Data Science Final Project

In this project, I aim to further the analysis for my arts and jobs paper. 
My main research question is: what drives growth, jobs or amenities? Do people follow jobs or amenities or both? 
Using regression analysis in the previous paper, I found that arts are a driver of growth. However, it is stronger in some places than others; for example more populated areas, and more educated areas. But what really is the underelying variables that would distinguish why arts are more influential in one places than in others? What variables would be more relevant in the jobs and amenities growth?

With over 50 control variables available, I would like to know which ones would help describe the changes better than only the main regression variables. 

In [1]:
%matplotlib inline
from collections import Counter
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN
import seaborn as sns

In [2]:
df = pd.read_csv("nomiss_scenes_data.csv")

We can list the control variables with the code below. And some variables of higher interest are listed on the table. 

In [4]:
list(df.columns.values)

['Unnamed: 0',
 'zip',
 'JobsT_1998',
 'ArtsN_1998',
 'ArtsW_1998',
 'JobsN_1998',
 'JobsW_1998',
 'ArtSupport_1998',
 'JobsT_1999',
 'ArtsN_1999',
 'ArtsW_1999',
 'JobsN_1999',
 'JobsW_1999',
 'ArtSupport_1999',
 'JobsT_2000',
 'ArtsN_2000',
 'ArtsW_2000',
 'JobsN_2000',
 'JobsW_2000',
 'ArtSupport_2000',
 'JobsT_2001',
 'ArtsN_2001',
 'ArtsW_2001',
 'JobsN_2001',
 'JobsW_2001',
 'ArtSupport_2001',
 'JobsT_2002',
 'ArtsN_2002',
 'ArtsW_2002',
 'JobsN_2002',
 'JobsW_2002',
 'ArtSupport_2002',
 'JobsT_2003',
 'ArtsN_2003',
 'ArtsW_2003',
 'JobsN_2003',
 'JobsW_2003',
 'ArtSupport_2003',
 'JobsT_2004',
 'ArtsN_2004',
 'ArtsW_2004',
 'JobsN_2004',
 'JobsW_2004',
 'ArtSupport_2004',
 'JobsT_2005',
 'ArtsN_2005',
 'ArtsW_2005',
 'JobsN_2005',
 'JobsW_2005',
 'ArtSupport_2005',
 'JobsT_2006',
 'ArtsN_2006',
 'ArtsW_2006',
 'JobsN_2006',
 'JobsW_2006',
 'ArtSupport_2006',
 'JobsT_2007',
 'ArtsN_2007',
 'ArtsW_2007',
 'JobsN_2007',
 'JobsW_2007',
 'ArtSupport_2007',
 'JobsT_2008',
 'ArtsN_2008


| Variable | Description  |
|------|------|
| Pop2000 | Population in 2000 |
| CrimeRate1999county | Crime Rate in 1999 | 
| Democ92 | Percent Democrats | 
|  pctba2k  | Percent Graduated Bachelor Degree in 2000|
| pctrental2k | Percent Rentals in 2000 | 
| pctmarried2k | Percent Married Households in 2000 | 
| pctwalk2k | Percent of people who walk in 2000 | 


| Variables | Scenes Dimension |
|------|------|
| esNAICS_INT11, esNAICS_AP11 | Traditionalistic |
| esNAICS_INT12, esNAICS_AP12 | Self_expressive |
| esNAICS_INT13, esNAICS_AP13 | Utilitarian |
| esNAICS_INT14, esNAICS_AP14 | Charismatic |
| esNAICS_INT15, esNAICS_AP15 | Egalitarian |
| esNAICS_INT21, esNAICS_AP21 | Neighborly |
| esNAICS_INT22, esNAICS_AP22 | Formality |
| esNAICS_INT23, esNAICS_AP23 | Exhibitionism |
| esNAICS_INT24, esNAICS_AP24 | Glamorous |
| esNAICS_INT25, esNAICS_AP25 | Transgressive |
| esNAICS_INT26, esNAICS_AP26 | Rational |
| esNAICS_INT31, esNAICS_AP31 | Locality |
| esNAICS_INT32, esNAICS_AP32 | State |
| esNAICS_INT33, esNAICS_AP33 | Corporateness |
| esNAICS_INT34, esNAICS_AP34 | Ethnicity |
| esNAICS_DP11 | Traditionalistic performance |
| esNAICS_DP12 | Self-expressive performance |

Besides these variables, I also have a number of establishments, and number of industries in each zip code. I could also get a list of individual industries present in each zip code from the raw data. And the Scenes data (my advisor's research), which coded each zip code on 15 scenes dimensions such as authenticity, corporativeness, etc. 

It would be specially interesting to do an analysis with the Scenes dimensions to see how the arts and jobs behave according to them. (And my advisor would like it too). 

I would like to do principal component analysis on the data, to see how the scenes dimensions group together for the "optimum" level of jobs by arts growth.

I also would like to try autoregression on the variables on themselves over time. 